Tokenize the text (e.g. use the nltk.word_tokenize() function in the NLTK package)
and compute the frequency of words. Then, plot the frequency distribution of words in each collection after the
removal of the stopwords: x-axis - word frequency (number of times a word appears in the collection); y-axis
-proportion of words with this frequency. Plot the distributions on a log-log scale. Does each plot look like a
power-law distribution? Are the two distributions similar or different?

Now compare the two collections more rigorously. Report the following properties of each
collection. Can you explain these differences based on the nature of the two collections? (20 points) (You can
use the nltk.pos tag() function of the NLTK package for part of speech tagging.)
(a) frequency of stopwords (percentage of the word occurrences that are stopwords.);
(b) percentage of capital letters;
(c) average number of characters per word;
(d) percentage of nouns, adjectives, verbs, adverbs, and pronouns;
(e) the top 10 nouns, top 10 verbs, and top 10 adjectives

We would like to summarize each document with a few words. However, picking the most
frequently used words in each document would be a bad idea, since they are more likely to appear in other
document as well. Instead, we pick the words with the highest TF-IDF weights in each document.
In this problem, term frequency (TF) and inverse document frequency (IDF) are defined as:
- *TF(t, d) = log(c(t, d) + 1)*
- *IDF(t) = 1 + log(N/k).*

c(t, d) is the frequency count of term t in doc d, N is the total number of documents in the collection, and k is
the document frequency of term t in the collection.
For each of the first 10 documents in the EHR collection, print out the 5 words that have the highest TF-IDF
weights.

As discussed in the class, TF-IDF is a common way to weight the terms in each document. It
can also be easily calculated from the inverted index, since TF can be obtained from the postings and IDF can
be summarized as a dictionary. Could you think of another weighting that cannot be calculated directly from
inverted index? What is the advantage of such a weighting?